In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import vaex as vx
import sys
import re
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import itertools
from tqdm.contrib.itertools import product
import pymorphy2
%matplotlib inline


tqdm.pandas()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/alexkon/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
#!pip3 install vaex
#!pip3 install autocorrect
#!pip3 install fuzzywuzzy
#!pip3 install python-Levenshtein
#!pip3 install pymorphy2

In [ ]:
# Объявление функции, очищающей выходные результаты тегов

In [13]:
def stops(x):
    x = x.split(' ')
    if x[-1] in stop_words:
        return 1
    return 0

stop_words = stopwords.words('russian')

In [ ]:
# Чтение входных данных

In [20]:
data = pd.read_csv('data_source.csv', index_col=0)
data.dropna(subset=['index'], inplace=True)

/opt/anaconda3/lib/python3.8/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [ ]:
words_clear = data.loc[data.source == 'words_clear'].copy()
double_words = data.loc[data.source == 'double_words'].copy()
triple_words = data.loc[data.source == 'triple_words'].copy()
words_nostop = data.loc[data.source == 'words_nostop'].copy()

In [43]:
source = pd.read_csv('source_req.csv', index_col=0)
source = list(source['0'].unique())

In [ ]:
# Формирование списка ключевых слов

### Запрос

In [58]:
word_phrase = 'платье с вырезом'

### ---------------------------------------

In [59]:
word_phrase = word_phrase.split(' ')
sign = 0
word_list = list(words_clear['index'])
key_words = []

#Подбор ключевых слов к каждому запросу
for word_input in word_phrase:
    if word_input in stop_words:
        if word_input == 'без':
            sign='-'
            continue
        else:
            sign='+'
            continue
    temp = {}

# Расчет похожести слов запроса с ключевыми словами
    for i in tqdm(range(len(word_list))):
        rat = fuzz.ratio(word_input, word_list[i])
        if rat>50:
            temp[word_list[i]] = rat
    compare = pd.DataFrame.from_dict(temp, orient='index').reset_index()
    compare.rename({0: 'simi'}, axis=1, inplace=True)
    compare = compare.merge(words_clear, how='left', on='index')

# Расчет метрики для выявления необходимого слова
    compare['metric'] = (0.01*compare['simi'])*np.log(compare['num'])
    compare.sort_values(by='metric', ascending=False, inplace=True)
    key_words.append(compare['index'].iloc[0])

100%|██████████| 123428/123428 [00:00<00:00, 452765.69it/s]


In [60]:
# Формирование списка наиболее используемых слов в одном запросе вместе с ключевыми

In [61]:
using = {}
top_head_dict = {1:10, 2:6, 3:3, 4:3}
# Составление списка наиболее используемых слов для каждого ключевого
for key_word in key_words:   
    temp = {}
    for i in tqdm(range(len(source))):
        if key_word in source[i]:
            x = source[i].split(' ')
            for j in x:
                if j in temp.keys():
                    temp[j] +=1
                else:
                    temp[j] = 1
    most_used = pd.DataFrame.from_dict(temp, orient='index').reset_index()
    most_used.rename({0:'use_num'}, axis=1, inplace=True)
    most_used.sort_values(by='use_num', ascending=False, inplace=True)
    most_used = most_used[~most_used['index'].isin(stop_words)]
    most_used = most_used.head(top_head_dict[len(key_words)])
    most_used = most_used.merge(words_nostop, how='left', on='index')
    using[key_word] = most_used

100%|██████████| 15148855/15148855 [00:04<00:00, 3159625.20it/s]


In [62]:
# Формирование словосочетаний-тэгов на основе совпадения и используемости с ключевыми словами

In [63]:
key_list = []
weight_list = []
matching_list = {}
metric_list = {}
match_size = 1

# Расчет весов для каждого ключевого слова. 
# Словосочетания составляются с помощью декартова произведения
for key_word in using.keys():  
    key_list.append(list(using[key_word]['index']))
    weight_list.append(list(using[key_word]['use_num']))
    match_size *= len(using[key_word])
# Расчет весов для словосочетаний из ключевых слов. 
weights = list(product(*weight_list))
max_weight = np.max(weights)
for i in range(len(weights)):
    weights[i] = np.sum(list(weights[i]))
step = 0

# Расчет метрик для словосочетаний и выбор наиболее релевантных
for elem in product(*key_list):
    cur_weight = 100*weights[step]/max_weight
    step+=1
# Словосочетания из двух слов
    for row in double_words.itertuples():
        t = fuzz.token_sort_ratio(row[1], str(' '.join(elem)))
        if t>50:
            metr = t*cur_weight*np.log(np.log(row[2]))
            if row[1] in matching_list.keys():
                if metr>metric_list[row[1]]:
                    metric_list[row[1]] = metr
            else:
                matching_list[row[1]] = 1
                metric_list[row[1]] = metr
# Словосочетания из трех слов
    for row in triple_words.itertuples():
        t = fuzz.token_sort_ratio(row[1], str(' '.join(elem)))
        if t>50:
            metr = t*cur_weight*np.log(np.log(row[2]))
            if row[1] in matching_list.keys():
                if metr>metric_list[row[1]]:
                    metric_list[row[1]] = metr
            else:
                matching_list[row[1]] = 1
                metric_list[row[1]] = metr

# Расчет финальной метрики
for x in matching_list.keys():
    matching_list[x] = matching_list[x]*metric_list[x]

# Отбор лучших кандидатов
matches = pd.DataFrame.from_dict(matching_list, orient='index').reset_index()
matches.rename({0:'num'}, axis=1, inplace=True)
matches.sort_values(by='num',  ascending=False, inplace=True)
matches = matches.head(50)
matches['flag'] = matches['index'].apply(stops)
matches = matches.loc[matches.flag == 0].copy()

  0%|          | 0/36 [00:00<?, ?it/s]

  0%|          | 0/36 [00:00<?, ?it/s]

In [64]:
# Формирование ответа
print('Запрос:')
print(' '.join(word_phrase))
print('\nТеги:')
list(matches.head(10)['index'])

Запрос:
платье с вырезом

Теги:


['платье с вырезом',
 'платье летнее',
 'вечернее платье',
 'платье теплое',
 'платье платье',
 'вязаное платье',
 'платье женское',
 'платье нарядное',
 'платье /',
 'платье demur']